In [9]:
import pandas as pd
import re
import requests
import certifi
from bs4 import BeautifulSoup

In [16]:
import json
import requests
from bs4 import BeautifulSoup as BS
url='https://api.bcra.gob.ar/estadisticas/v2.0/principalesvariables'
#url='https://www.bcra.gob.ar/'
#url='https://open.bymadata.com.ar/#/dashboard'
response=requests.get(url, verify=False)
#response=json(response.json)
#soup=BS(response.text, 'html.parser')
#soup
a=json.loads(response.text)
a
#0: Reservas
#3: T pol
#2: TC


c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'open.bymadata.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'<!DOCTYPE html>\n<html>\n  <head>\n    <meta charset="utf-8" />\n\n    <title>BYMADATA - Cotizaciones en Tiempo Real | BYMA Bolsas y Mercados Argentinos</title>\n\n    <meta name="description" content="BYMA te acerca la informaciÃ³n mÃ¡s relevante del Mercado de Capitales argentino en un solo lugar." />\n\n    <meta name="facebook-domain-verification" content="x9zbmyu0rbwsxvklczmdu6ujdbq5s8" />\n\n    <!-- base url -->\n    <base href="/" />\n\n    <meta\n      name="viewport"\n      content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no"\n    />\n    <link\n      href="https://fonts.googleapis.com/icon?family=Material+Icons"\n      rel="stylesheet"\n    />\n    <!-- #CSS Links -->\n    <!-- Basic Styles -->\n    <link\n      rel="stylesheet"\n      type="text/css"\n      media="screen"\n      href="assets/css/bootstrap.min.css"\n    />\n    <link\n      rel="stylesheet"\n      type="text/css"\n      media="screen"\n      href="assets/css/font-awesome.min.

In [19]:
_=0
for i in a['results']:
    print(_,' - ',i['descripcion'])
    _+=1

0  -  Reservas Internacionales del BCRA (en millones de dólares - cifras provisorias sujetas a cambio de valuación)
1  -  Tipo de Cambio Minorista ($ por USD) Comunicación B 9791 - Promedio vendedor
2  -  Tipo de Cambio Mayorista ($ por USD) Comunicación A 3500 - Referencia
3  -  Tasa de Política Monetaria (en % n.a.)
4  -  BADLAR en pesos de bancos privados (en % n.a.)
5  -  TM20 en pesos de bancos privados (en % n.a.)
6  -  Tasas de interés de las operaciones de pase activas para el BCRA, a 1 día de plazo (en % n.a.)
7  -  Tasas de interés de las operaciones de pase pasivas para el BCRA, a 1 día de plazo (en % n.a.)
8  -  Tasas de interés por préstamos entre entidades financiera privadas (BAIBAR) (en % n.a.)
9  -  Tasas de interés por depósitos a 30 días de plazo en entidades financieras (en % n.a.)
10  -  Tasa de interés de préstamos por adelantos en cuenta corriente
11  -  Tasa de interés de préstamos personales
12  -  Base monetaria - Total (en millones de pesos)
13  -  Circulació

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Configuración del WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecuta el navegador en modo headless (sin interfaz gráfica)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Inicia el WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Abre la página
url = 'https://open.bymadata.com.ar/#/dashboard'
driver.get(url)

# Espera a que se cargue la página (podrías necesitar ajustar el tiempo de espera)
driver.implicitly_wait(10)

# Extrae el HTML de la página
html = driver.page_source

# Puedes hacer más cosas con el WebDriver, como encontrar elementos
# ejemplo_element = driver.find_element(By.XPATH, 'XPATH_DEL_ELEMENTO')

# Cierra el WebDriver
driver.quit()

# Imprime el HTML extraído
print(html)


<html><head>
    <meta charset="utf-8">

    <title>BYMADATA - Cotizaciones en Tiempo Real | BYMA Bolsas y Mercados Argentinos</title>

    <meta name="description" content="BYMA te acerca la información más relevante del Mercado de Capitales argentino en un solo lugar.">

    <meta name="facebook-domain-verification" content="x9zbmyu0rbwsxvklczmdu6ujdbq5s8">

    <!-- base url -->
    <base href="/">

    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    <link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet">
    <!-- #CSS Links -->
    <!-- Basic Styles -->
    <link rel="stylesheet" type="text/css" media="screen" href="assets/css/bootstrap.min.css">
    <link rel="stylesheet" type="text/css" media="screen" href="assets/css/font-awesome.min.css">

    <!-- SmartAdmin Styles : Caution! DO NOT change the order -->
    <link rel="stylesheet" type="text/css" media="screen" href="assets/css/smarta

In [238]:
import pandas as pd # 0.23.0
import requests     # 2.19.1
import io

url = 'http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?csv.x=yes'

payload = {
    'Datefrom'   : '01/Jan/2000',
    #'Dateto'     : '01/Oct/2018',
    'SeriesCodes': 'IUDBEDR',
    'CSVF'       : 'TN',
    'UsingCodes' : 'Y',
    'VPD'        : 'Y',
    'VFD'        : 'N'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/54.0.2840.90 '
                  'Safari/537.36'
}
response = requests.get(url, params=payload, headers=headers)
tas = pd.read_csv(io.BytesIO(response.content),names=['Fecha','Tasa'],skiprows=1)
tas['Fecha']=pd.to_datetime(tas['Fecha'], format='%d %b %Y')#.dt.strftime('%d-%m-%Y')
tas.set_index('Fecha',inplace=True)
#tas=tas.resample('M').last()
tas_t=tas['Tasa'].iloc[-1]
tas_t1=tas['Tasa'].iloc[-2]
#Metrica

url='https://www.ons.gov.uk/generator?format=csv&uri=/economy/inflationandpriceindices/timeseries/l55o/mm23'
response=requests.get(url)
data = io.StringIO(response.text)
inf = pd.read_csv(data,skiprows=316,names=['Fecha','Inflacion'])
inf.columns=['Fecha','Inflacion']
inf['Fecha']=pd.to_datetime(inf['Fecha'], format='%Y %b')#.dt.strftime('%d-%m-%Y')
#metrica

url='https://www.ons.gov.uk/generator?format=csv&uri=/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms'
response=requests.get(url)
data = io.StringIO(response.text)
une = pd.read_csv(data,skiprows=621,names=['Fecha','Inflacion'])
une.columns=['Fecha','Desempleo']
une['Fecha']=pd.to_datetime(une['Fecha'], format='%Y %b')#.dt.strftime('%d-%m-%Y')

In [240]:
#tas.index=pd.to_datetime(tas.index)
tas.index[-1].strftime('%b')

'May'

In [224]:
tas

,Fecha,Tasa
0,04-01-2000,5.50
1,05-01-2000,5.50
2,06-01-2000,5.50
3,07-01-2000,5.50
4,10-01-2000,5.50
...,...,...
6160,22-05-2024,5.25
6161,23-05-2024,5.25
6162,24-05-2024,5.25
6163,28-05-2024,5.25


In [216]:
_.head(12)

,Fecha,Tasa
0,04 Jan 2000,5.50
1,05 Jan 2000,5.50
2,06 Jan 2000,5.50
3,07 Jan 2000,5.50
4,10 Jan 2000,5.50
5,11 Jan 2000,5.50
6,12 Jan 2000,5.50
7,13 Jan 2000,5.75
8,14 Jan 2000,5.75
9,17 Jan 2000,5.75


In [178]:
from io import StringIO
ulr='https://www.ons.gov.uk/generator?format=csv&uri=/economy/inflationandpriceindices/timeseries/l55o/mm23'
response=requests.get(ulr)
data = StringIO(response.text)
inf = pd.read_csv(data,skiprows=316,names=['Fecha','Inflacion'])
inf.columns=['Fecha','Inflacion']
inf['Fecha']=pd.to_datetime(inf['Fecha'], format='%Y %b')
inf

,Fecha,Inflacion
0,2000-01-01,1.2
1,2000-02-01,1.2
2,2000-03-01,1.0
3,2000-04-01,0.9
4,2000-05-01,1.0
...,...,...
287,2023-12-01,4.2
288,2024-01-01,4.2
289,2024-02-01,3.8
290,2024-03-01,3.8


In [187]:
url='https://www.ons.gov.uk/generator?format=csv&uri=/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms'
response=requests.get(url)
data = StringIO(response.text)
une = pd.read_csv(data,skiprows=621,names=['Fecha','Inflacion'])
une.columns=['Fecha','Desempleo']
une['Fecha']=pd.to_datetime(une['Fecha'], format='%Y %b')
une

,Fecha,Desempleo
0,2000-01-01,5.8
1,2000-02-01,5.8
2,2000-03-01,5.7
3,2000-04-01,5.6
4,2000-05-01,5.5
...,...,...
285,2023-10-01,3.9
286,2023-11-01,3.8
287,2023-12-01,4.0
288,2024-01-01,4.2


In [5]:
import requests
import json
import pandas as pd
# URL del endpoint de la API
url = "https://www.presupuestoabierto.gob.ar/api/v1/credito?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Credito vigente por jurisdiccion",
    "columns": [
        "impacto_presupuestario_mes",
        "jurisdiccion_desc",
        "finalidad_desc",
        "fuente_financiamiento_desc",
        "ubicacion_geografica_desc",
        "credito_vigente"
    ]
}

# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))


with open('presupuestos.csv', 'w', encoding='utf-8') as file:
    file.write(response.text)
df=pd.read_csv('presupuestos.csv',encoding='utf-8')
df

ConnectTimeout: HTTPSConnectionPool(host='www.presupuestoabierto.gob.ar', port=443): Max retries exceeded with url: /api/v1/credito?format=csv (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002A336871390>, 'Connection to www.presupuestoabierto.gob.ar timed out. (connect timeout=None)'))

In [8]:
import pandas as pd
import json
provincias={'Ciudad Autónoma de Buenos Aires':'Capital Federal',
 'Provincia de Buenos Aires':'Buenos Aires',
 'Provincia de Catamarca':'Catamarca',
 'Provincia de Corrientes':'Corrientes',
 'Provincia de Córdoba':'Córdoba',
 'Provincia de Entre Ríos':'Entre Ríos',
 'Provincia de Formosa':'Formosa',
 'Provincia de Jujuy':'Jujuy',
 'Provincia de La Pampa':'La Pampa',
 'Provincia de La Rioja':'La Rioja',
 'Provincia de Mendoza':'Mendoza',
 'Provincia de Misiones':'Misiones',
 'Provincia de Río Negro':'Río Negro',
 'Provincia de Salta':'Salta',
 'Provincia de San Juan':'San Juan',
 'Provincia de San Luis':'San Luis',
 'Provincia de Santa Cruz':'Santa Cruz',
 'Provincia de Santa Fe':'Santa Fe',
 'Provincia de Santiago del Estero':'Santiago del Estero',
 'Provincia de Tierra del Fuego, Antártida e Islas del Atlántico Sur':'Tierra del Fuego, Antártida e Islas del Atlántico Sur',
 'Provincia de Tucumán':'Tucumán',
 'Provincia del Chaco':'Chaco',
 'Provincia del Chubut':'Chubut',
 'Provincia del Neuquén':'Neuquén'}
df=pd.read_csv('presupuestos.csv',encoding='utf-8')

df1=df.groupby(by=['ubicacion_geografica_desc']).sum()#.reset_index()
df1['%']=100*df1['credito_vigente']/df1['credito_vigente'].sum()
extras=df1.iloc[0:4]
df1=df1.drop(index=['Binacional','No Clasificado','Interprovincial','Nacional'],)
df1=df1.reset_index()
df1['ubicacion_geografica_desc'] = df1['ubicacion_geografica_desc'].replace(provincias)


geo=json.load(open('provincias.geojson', encoding='utf-8'))
provincias_geo_df = pd.json_normalize(geo['features'])
#provincias_geo_df['properties.nombre'] = 'Provincia de ' + provincias_geo_df['properties.nombre']

merged_df = provincias_geo_df.merge(df1, left_on='properties.nombre', right_on='ubicacion_geografica_desc')
merged_df

C:\Users\feder\AppData\Local\Temp\ipykernel_7100\855307051.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df1=df.groupby(by=['ubicacion_geografica_desc']).sum()#.reset_index()


,type,geometry.type,geometry.coordinates,properties.id,properties.nombre,properties.nombre_completo,properties.fuente,properties.categoria,properties.centroide.lon,properties.centroide.lat,properties.iso_id,properties.iso_nombre,ubicacion_geografica_desc,impacto_presupuestario_mes,credito_vigente,%
0,Feature,Polygon,"[[[-70.393406682, -36.156241237], [-70.3949855...",58,Neuquén,Provincia del Neuquén,IGN,Provincia,-70.119897,-38.641983,AR-Q,Neuquén,Neuquén,833,4.187527e+05,0.759584
1,Feature,Polygon,"[[[-67.055470113, -31.856052635], [-67.0059302...",74,San Luis,Provincia de San Luis,IGN,Provincia,-66.025231,-33.761104,AR-D,San Luis,San Luis,720,4.359788e+05,0.790831
2,Feature,Polygon,"[[[-61.046389631, -27.9980009399999], [-58.884...",82,Santa Fe,Provincia de Santa Fe,IGN,Provincia,-60.950687,-30.708823,AR-S,Santa Fe,Santa Fe,967,1.664028e+06,3.018416
3,Feature,Polygon,"[[[-68.521356179, -27.8179169699999], [-68.510...",46,La Rioja,Provincia de La Rioja,IGN,Provincia,-67.181758,-29.684937,AR-F,La Rioja,La Rioja,701,3.883016e+05,0.704349
4,Feature,Polygon,"[[[-68.5053654399999, -25.168509816], [-67.864...",10,Catamarca,Provincia de Catamarca,IGN,Provincia,-66.947897,-27.335954,AR-K,Catamarca,Catamarca,686,4.199475e+05,0.761752
5,Feature,Polygon,"[[[-65.3591542509999, -26.0594376889999], [-65...",90,Tucumán,Provincia de Tucumán,IGN,Provincia,-65.364766,-26.948284,AR-T,Tucumán,Tucumán,855,1.022367e+06,1.854494
6,Feature,Polygon,"[[[-62.3243395499999, -24.1231840889999], [-62...",22,Chaco,Provincia del Chaco,IGN,Provincia,-60.765116,-26.386987,AR-H,Chaco,Chaco,820,6.173304e+05,1.119789
7,Feature,Polygon,"[[[-62.312005272, -22.4866595899999], [-62.290...",34,Formosa,Provincia de Formosa,IGN,Provincia,-59.932190,-24.895087,AR-P,Formosa,Formosa,685,3.014772e+05,0.546856
8,Feature,MultiPolygon,"[[[[-67.593607919, -46.017442156], [-67.585965...",78,Santa Cruz,Provincia de Santa Cruz,IGN,Provincia,-69.955762,-48.815547,AR-Z,Santa Cruz,Santa Cruz,843,4.151853e+05,0.753113
9,Feature,MultiPolygon,"[[[[-67.75056885, -41.99998806], [-65.06925398...",26,Chubut,Provincia del Chubut,IGN,Provincia,-68.526736,-43.788627,AR-U,Chubut,Chubut,862,4.780174e+05,0.867086


In [171]:
provincias={'Ciudad Autónoma de Buenos Aires':'Capital Federal',
 'Provincia de Buenos Aires':'Buenos Aires',
 'Provincia de Catamarca':'Catamarca',
 'Provincia de Corrientes':'Corrientes',
 'Provincia de Córdoba':'Córdoba',
 'Provincia de Entre Ríos':'Entre Ríos',
 'Provincia de Formosa':'Formosa',
 'Provincia de Jujuy':'Jujuy',
 'Provincia de La Pampa':'La Pampa',
 'Provincia de La Rioja':'La Rioja',
 'Provincia de Mendoza':'Mendoza',
 'Provincia de Misiones':'Misiones',
 'Provincia de Río Negro':'Río Negro',
 'Provincia de Salta':'Salta',
 'Provincia de San Juan':'San Juan',
 'Provincia de San Luis':'San Luis',
 'Provincia de Santa Cruz':'Santa Cruz',
 'Provincia de Santa Fe':'Santa Fe',
 'Provincia de Santiago del Estero':'Santiago del Estero',
 'Provincia de Tierra del Fuego, Antártida e Islas del Atlántico Sur':'Tierra del Fuego, Antártida e Islas del Atlántico Sur',
 'Provincia de Tucumán':'Tucumán',
 'Provincia del Chaco':'Chaco',
 'Provincia del Chubut':'Chubut',
 'Provincia del Neuquén':'Neuquén'}
df=pd.read_csv('presupuestos.csv',encoding='utf-8')

df1=df.groupby(by=['ubicacion_geografica_desc']).sum()#.reset_index()
df1['%']=100*df1['credito_vigente']/df1['credito_vigente'].sum()
extras=df1.iloc[0:4]
df1=df1.drop(index=['Binacional','No Clasificado','Interprovincial','Nacional'],)
df1=df1.reset_index()
df1['ubicacion_geografica_desc'] = df1['ubicacion_geografica_desc'].replace(provincias)


geo=json.load(open('ProvinciasArgentina.geojson', encoding='utf-8'))
provincias_geo_df = pd.json_normalize(geo['features'])
#provincias_geo_df['properties.nombre'] = 'Provincia de ' + provincias_geo_df['properties.nombre']

merged_df = provincias_geo_df.merge(df1, left_on='properties.nombre', right_on='ubicacion_geografica_desc')

fig = px.choropleth_mapbox(
    merged_df,
    geojson=geo,
    locations='ubicacion_geografica_desc',
    featureidkey='properties.nombre',
    color='%', # Asegúrate de que esta columna contiene el gasto fiscal
    hover_name='properties.nombre',
    custom_data=['properties.nombre','credito_vigente','%'],
    color_continuous_scale='magma',
                           mapbox_style= "carto-positron" , # formatos de diseño del mapa : "carto-positron", "carto-positron",   "white-bg",
                           zoom=2.6, center = {"lat": -38.40, "lon": -63.60},
                           opacity=1,
                           labels={'promedio acessos por cada 100 hogares':'acceso a internet'},
                           color_discrete_sequence=["blue"],
                          )
fig.update_traces(
    marker_line_width=1.5,
    marker_line_color='black',
    hovertemplate="<br>".join([
        "<b>%{customdata[0]}</b>",  # Asegúrate de cerrar la etiqueta <b> correctamente aquí
        "Presupuesto Brindado: %{customdata[1]}",
        "Proporción en relación al presupuesto total: %{customdata[2]:.2f}%"
    ])
)
# Actualizar el layout del mapa
#fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    mapbox=dict(
        center={"lat": -38.4161, "lon": -63.6167},
        style="white-bg",
        zoom=2.65,
        layers=[
            dict(
                below='traces',
                type='fill',
                source=geo,
                color="lightblue"
            )
        ]
    ),
    showlegend=False,
    margin=dict(t=0, b=0, l=0, r=0),
    geo=dict(
        showland=False,  # No mostrar etiquetas de países
        showcountries=False,  # No mostrar bordes de países
    )
)

fig.show()

C:\Users\feder\AppData\Local\Temp\ipykernel_14436\3697934728.py:27: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [172]:
import json
import folium

# Leer el archivo JSON
with open('provincias.json', 'r') as file:
    data = json.load(file)

# Crear el mapa centrado en Argentina
m = folium.Map(location=[-38.4161, -63.6167], zoom_start=4)

# Agregar las provincias al mapa
for feature in data['features']:
    nombre = feature['properties']['nombre']
    coords = feature['geometry']['coordinates']
    folium.Polygon(
        locations=[(lat, lon) for lon, lat in coords[0]],
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=nombre
    ).add_to(m)

# Guardar el mapa en un archivo HTML
map_path = "mapa_argentina.html"
m.save(map_path)

map_path


'mapa_argentina.html'

In [83]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/pef?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"ejercicio_presupuestario",
"trimestre",
"finalidad_id",
"finalidad_desc",
"funcion_id",
"funcion_desc",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"programa_id",
"programa_desc",
"subprograma_id",
"subprograma_desc",
"proyecto_id",
"proyecto_desc",
"actividad_id",
"actividad_desc",
"obra_id",
"obra_desc",
"medicion_fisica_id",
"medicion_fisica_desc",
"tipo_medicion_fisica",
"unidad_medida_id",
"unidad_medida_desc",
"ubicacion_geografica_id",
"ubicacion_geografica_desc",
"totalizador_avance_fisico",
"programacion_inicial_DA",
"programacion_inicial_ajustada",
"programacion_anual_vig_cierre",
"programacion_anual_vig_trim",
"programacion_trim",
"programacion_acumulada_trim",
"ejecutado_vigente_trim",
"ejecutado_acumulado_trim",
"ejecutado_cierre_acum_trim",
"ejecucion_anual_de_cierre",
"tipo_causa_desvio",
"causa_desvio",
"causa_desvio_detalle",
"causa_desvio_comentario",
"porc_desvio_acum_trim",
"ultima_actualizacion_fecha"
],
    "ejercicios":[2024]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))


In [84]:
response

<Response [200]>

In [85]:
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('credito_vigente_por_jurisdiccion.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    print("Datos guardados en credito_vigente_por_jurisdiccion.csv")
else:
    print(f"Error {response.status_code}: {response.text}")

Datos guardados en credito_vigente_por_jurisdiccion.csv


In [89]:
df=pd.read_csv('credito_vigente_por_jurisdiccion.csv')
df.columns#.iloc[1]

Index(['ejercicio_presupuestario', 'trimestre', 'finalidad_id',
       'finalidad_desc', 'funcion_id', 'funcion_desc', 'sector_id',
       'sector_desc', 'subsector_id', 'subsector_desc', 'caracter_id',
       'caracter_desc', 'jurisdiccion_id', 'jurisdiccion_desc',
       'subjurisdiccion_id', 'subjurisdiccion_desc', 'entidad_id',
       'entidad_desc', 'servicio_id', 'servicio_desc', 'programa_id',
       'programa_desc', 'subprograma_id', 'subprograma_desc', 'proyecto_id',
       'proyecto_desc', 'actividad_id', 'actividad_desc', 'obra_id',
       'obra_desc', 'medicion_fisica_id', 'medicion_fisica_desc',
       'tipo_medicion_fisica', 'unidad_medida_id', 'unidad_medida_desc',
       'ubicacion_geografica_id', 'ubicacion_geografica_desc',
       'totalizador_avance_fisico', 'programacion_inicial_DA',
       'programacion_inicial_ajustada', 'programacion_anual_vig_cierre',
       'programacion_anual_vig_trim', 'programacion_trim',
       'programacion_acumulada_trim', 'ejecutado_vige

In [91]:
df['subjurisdiccion_desc'].unique()

array(['Poder Legislativo Nacional', 'Poder Judicial de la Nación',
       'Ministerio de Justicia', 'Procuración del Tesoro de la Nación',
       'Secretaría Legal y Técnica', 'Secretaría de Prensa',
       'Jefatura de Gabinete de Ministros', 'Ministerio de Economía',
       'Ministerio de Infraestructura',
       'Ministerio de Relaciones Exteriores, Comercio Internacional y Culto',
       'Ministerio del Interior', 'Presidencia de la Nación',
       'Ministerio de Defensa',
       'Estado Mayor General del Ejército Argentino',
       'Estado Mayor General de la Armada Argentina',
       'Estado Mayor General de la Fuerza Aérea Argentina',
       'Estado Mayor Conjunto de las Fuerzas Armadas (EMCO)',
       'Ministerio de Seguridad', 'Policía Federal Argentina',
       'Gendarmería Nacional', 'Prefectura Naval Argentina',
       'Policía de Seguridad Aeroportuaria',
       'Servicio Penitenciario Federal', 'Ministerio de Salud',
       'Ministerio de Capital Humano',
       'Secreta

In [54]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/transversal_financiero?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"ejercicio_presupuestario",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"programa_id",
"programa_desc",
"subprograma_id",
"subprograma_desc",
"proyecto_id",
"proyecto_desc",
"actividad_id",
"actividad_desc",
"obra_id",
"obra_desc",
"finalidad_id",
"finalidad_desc",
"funcion_id",
"funcion_desc",
"fuente_financiamiento_id",
"fuente_financiamiento_desc",
"ubicacion_geografica_id",
"ubicacion_geografica_desc",
"inciso_id",
"inciso_desc",
"principal_id",
"principal_desc",
"parcial_id",
"parcial_desc",
"subparcial_id",
"subparcial_desc",
"etiqueta_id",
"etiqueta_desc",
"ponderacion_credito",
"codigo_bapin_id",
"codigo_bapin_desc",
"credito_inicial",
"credito_inicial_ponderado",
"credito_vigente",
"credito_vigente_ponderado",
"credito_ejecutado",
"credito_ejecutado_ponderado"
],
    
    "ejercicios":[2024]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('transversal_financiero.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    df=pd.read_csv('transversal_financiero.csv')
df.iloc[1]

In [78]:
df['subjurisdiccion_desc'].unique()

array(['Poder Legislativo Nacional', 'Poder Judicial de la Nación',
       'Ministerio de Justicia', 'Procuración del Tesoro de la Nación',
       'Secretaría Legal y Técnica', 'Secretaría de Prensa',
       'Jefatura de Gabinete de Ministros', 'Ministerio de Economía',
       'Ministerio de Infraestructura',
       'Ministerio de Relaciones Exteriores, Comercio Internacional y Culto',
       'Ministerio del Interior', 'Presidencia de la Nación',
       'Ministerio de Defensa',
       'Estado Mayor General del Ejército Argentino',
       'Estado Mayor General de la Armada Argentina',
       'Estado Mayor General de la Fuerza Aérea Argentina',
       'Estado Mayor Conjunto de las Fuerzas Armadas (EMCO)',
       'Ministerio de Seguridad', 'Policía Federal Argentina',
       'Gendarmería Nacional', 'Prefectura Naval Argentina',
       'Policía de Seguridad Aeroportuaria',
       'Servicio Penitenciario Federal', 'Ministerio de Salud',
       'Ministerio de Capital Humano',
       'Secreta

In [79]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/credito?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"impacto_presupuestario_fecha",
"impacto_presupuestario_anio",
"impacto_presupuestario_mes",
"ejercicio_presupuestario",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"programa_id",
"programa_desc",
"subprograma_id",
"subprograma_desc",
"proyecto_id",
"proyecto_desc",
"actividad_id",
"actividad_desc",
"obra_id",
"obra_desc",
"finalidad_id",
"finalidad_desc",
"funcion_id",
"funcion_desc",
"inciso_id",
"inciso_desc",
"principal_id",
"principal_desc",
"parcial_id",
"parcial_desc",
"subparcial_id",
"subparcial_desc",
"clasificador_economico_8_digitos_id",
"clasificador_economico_8_digitos_desc",
"fuente_financiamiento_id",
"fuente_financiamiento_desc",
"ubicacion_geografica_id",
"ubicacion_geografica_desc",
"unidad_ejecutora_id",
"unidad_ejecutora_desc",
"prestamo_externo_id",
"prestamo_externo_desc",
"codigo_bapin_id",
"codigo_bapin_desc",
"credito_presupuestado",
"credito_vigente",
"credito_comprometido",
"credito_devengado",
"credito_pagado",
"ultima_actualizacion_fecha"
],
    
    "ejercicios":[2023]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('credito.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    df=pd.read_csv('credito.csv')
df.iloc[1]

ejercicio_presupuestario                                                      2024
trimestre                                                                        1
finalidad_id                                                                     1
finalidad_desc                                        ADMINISTRACION GUBERNAMENTAL
funcion_id                                                                       1
funcion_desc                                                           Legislativa
sector_id                                                                        1
sector_desc                                  Sector Público Nacional no Financiero
subsector_id                                                                     1
subsector_desc                                             Administración Nacional
caracter_id                                                                      1
caracter_desc                                               Administración Central
juri

In [ ]:
url = "https://www.presupuestoabierto.gob.ar/api/v1/recurso?format=csv"

# Cabeceras de la solicitud
headers = {
    'Authorization': 'ee7f6d62-90ad-4a31-8db0-844acf12ee27',  # Reemplaza con tu token
    'Content-Type': 'application/json'
}

# Datos de la solicitud
data = {
    "title": "Programación y ejecución física del primer trimestre",
    
    "columns": [
"impacto_presupuestario_fecha",
"impacto_presupuestario_anio",
"impacto_presupuestario_mes",
"ejercicio_presupuestario",
"sector_id",
"sector_desc",
"subsector_id",
"subsector_desc",
"caracter_id",
"caracter_desc",
"jurisdiccion_id",
"jurisdiccion_desc",
"subjurisdiccion_id",
"subjurisdiccion_desc",
"entidad_id",
"entidad_desc",
"servicio_id",
"servicio_desc",
"tipo_id",
"tipo_desc",
"clase_id",
"clase_desc",
"concepto_id",
"concepto_desc",
"subconcepto_id",
"subconcepto_desc",
"fuente_financiamiento_id",
"fuente_financiamiento_desc",
"clasificador_economico_8_digitos_id",
"clasificador_economico_8_digitos_desc",
"prestamo_externo_id",
"prestamo_externo_desc",
"recurso_inicial",
"recurso_vigente",
"recurso_ingresado_percibido",
"ultima_actualizacion_fecha"
],
    
    "ejercicios":[2023]
}


# Realizar la solicitud POST a la API
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    # Guardar la respuesta directamente en un archivo CSV
    with open('recurso.csv', 'w', encoding='utf-8') as file:
        file.write(response.text)
    df=pd.read_csv('recurso.csv')
df.iloc[1]

,Titulos Publicos,Titulos Publicos-Moneda Nacional,Deuda no ajustable por CER,Deuda ajustable por CER,Titulos Publicos-Moneda Extranjera,Letras,Letras-Moneda Nacional,Letras-Moneda Extranjera,Prestamos,Total Deuda Bruta,Otros
2019-01-31,216771.7,43886.0,23231.6,20654.4,172870.8,35631.3,14952.1,20679.2,61442.8,335661.3,21815.5
2019-02-28,219175.7,44347.6,23465.5,20882.1,174813.2,35102.3,14451.3,20651.0,61420.0,334477.0,18779.0
2019-03-31,213330.2,41501.1,20538.2,20962.9,171814.3,34653.1,14081.6,20571.5,60737.9,324898.3,16177.1
2019-04-30,212709.8,43277.1,21581.7,21695.4,169417.9,33999.5,14259.7,19739.8,71608.7,334322.9,16004.9
2019-05-31,210302.7,43454.1,21305.9,22148.2,166833.8,34358.7,14934.9,19423.8,70448.5,329930.6,14820.7
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31,263424.7,85520.0,43313.8,42206.2,177890.7,10974.9,5097.1,5877.8,82239.7,370673.4,14034.1
2024-01-31,272907.6,93370.4,43288.4,50082.0,179523.1,10022.5,4175.6,5846.9,84237.9,381271.9,14103.9
2024-02-29,280981.2,109034.4,38836.7,70197.7,171932.5,8073.5,2212.1,5861.4,83198.7,386536.8,14283.4
2024-03-31,300792.8,128816.9,16342.5,112474.4,171961.4,8252.5,2391.1,5861.4,82788.8,403043.7,11209.6


In [126]:

# Descargar el archivo
response = requests.get(url)
response

<Response [200]>

Unnamed: 2     335661.3
Unnamed: 3       334477
Unnamed: 4     324898.3
Unnamed: 5     334322.9
Unnamed: 6     329930.6
                 ...   
Unnamed: 61    370673.4
Unnamed: 62    381271.9
Unnamed: 63    386536.8
Unnamed: 64    403043.7
Unnamed: 65    414056.1
Name: A- DEUDA BRUTA ( I + II  + III), Length: 64, dtype: object

Unnamed: 2     61442.8
Unnamed: 3     61420.0
Unnamed: 4     60737.9
Unnamed: 5     71608.7
Unnamed: 6     70448.5
                ...   
Unnamed: 61    82239.7
Unnamed: 62    84237.9
Unnamed: 63    83198.7
Unnamed: 64    82788.8
Unnamed: 65    80669.4
Name: PRÉSTAMOS, Length: 64, dtype: object

In [ ]:
columnas_originales=["A- DEUDA BRUTA ( I + II  + III)",
"nan",
"B- DEUDA BRUTA (EXCLUIDA LA ELEGIBLE PENDIENTE DE REESTRUCTURACIÓN) ( I + II )",
"nan",
"nan",
"I- DEUDA EN SITUACIÓN DE PAGO NORMAL",
"nan",
"        MEDIANO Y LARGO PLAZO",
"nan",
"TÍTULOS PÚBLICOS1",
"nan",
"TP-Moneda nacional",
"Deuda no ajustable por CER",
"BONO CONSOLIDADO 2089",
"BOCON",
"BONAR",
"BONTE",
"BOGATO",
"BOTAPO",
"BONO PGN",
"nan",
"Deuda ajustable por CER",
"BOCON",
"BONCER",
"BOGATO",
"CUASIPAR/$+CER/3,31%/2045",
"DISCOUNT/$+CER/5,83%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/$+CER/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
" - Moneda extranjera ",
"Deuda en dólares estadounidenses",
"LETRA BCRA FDA",
"LETRA BCRA FOI",
"LETRA BCRA",
"BONAR",
"BIRAD",
"BONO R.A. U$S",
"BONO GLOBAL U$S",
"DISCOUNT/U$S/8,28%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/U$S/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en euros",
"BIRAE",
"BONO GLOBAL EUR",
"DISCOUNT/EUR/7,82%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/EUR/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en yenes",
"DISCOUNT/JPY/4,33%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/JPY/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en Franco suizo",
"BIRAF",
"nan",
" - Amparos y excepciones",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA CMEA",
"LETES $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETES - U$S",
"LETRA MENDOZA - U$S",
"LETRAS EN GARANTÍA - U$S",
"nan",
"PRÉSTAMOS",
"nan",
"PRÉSTAMOS GARANTIZADOS",
"nan",
"ORGANISMOS INTERNACIONALES",
"BCIE",
"BIRF",
"BID",
"FONPLATA",
"FIDA",
"CAF",
"OFID",
"FMI",
"BEI",
"nan",
"ORGANISMOS OFICIALES",
"CLUB DE PARIS",
"OTROS BILATERALES",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARÉ 2019 - $",
"PAGARÉ 2021 - CAMMESA - U$S",
"PAGARÉ 2038 - UCP",
"nan",
"AVALES",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Extraordinarios",
"nan",
"        CORTO PLAZO (1)",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Ordinarios",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARE CUT",
"nan",
"AVALES",
"nan",
"TÍTULOS PÚBLICOS2",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA FFRH",
"LETRA FFSIT",
"LETRA SRT",
"LETRA ANSES",
"LETES - $",
"LETRA FFDP",
"LETRA FGS",
"LETRA - $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETRA FGS - U$S"]

In [ ]:
#TÍTULOS PÚBLICOS  =  Titpub1+titpub2
#same for letras del tesoro
nuevas_col={"A- DEUDA BRUTA ( I + II  + III)":'Total Deuda',
"nan",
"B- DEUDA BRUTA (EXCLUIDA LA ELEGIBLE PENDIENTE DE REESTRUCTURACIÓN) ( I + II )",
"nan",
"nan",
"I- DEUDA EN SITUACIÓN DE PAGO NORMAL",
"nan",
"        MEDIANO Y LARGO PLAZO",
"nan",
"TÍTULOS PÚBLICOS",
"nan",
" - Moneda nacional",
"Deuda no ajustable por CER",
"BONO CONSOLIDADO 2089",
"BOCON",
"BONAR",
"BONTE",
"BOGATO",
"BOTAPO",
"BONO PGN",
"nan",
"Deuda ajustable por CER",
"BOCON",
"BONCER",
"BOGATO",
"CUASIPAR/$+CER/3,31%/2045",
"DISCOUNT/$+CER/5,83%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/$+CER/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
" - Moneda extranjera ",
"Deuda en dólares estadounidenses",
"LETRA BCRA FDA",
"LETRA BCRA FOI",
"LETRA BCRA",
"BONAR",
"BIRAD",
"BONO R.A. U$S",
"BONO GLOBAL U$S",
"DISCOUNT/U$S/8,28%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/U$S/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en euros",
"BIRAE",
"BONO GLOBAL EUR",
"DISCOUNT/EUR/7,82%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/EUR/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en yenes",
"DISCOUNT/JPY/4,33%/2033",
"- Canje 2005",
"- Canje 2010",
"PAR/JPY/T.FIJA/2038",
"- Canje 2005",
"- Canje 2010",
"nan",
"Deuda en Franco suizo",
"BIRAF",
"nan",
" - Amparos y excepciones",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA CMEA",
"LETES $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETES - U$S",
"LETRA MENDOZA - U$S",
"LETRAS EN GARANTÍA - U$S",
"nan",
"PRÉSTAMOS",
"nan",
"PRÉSTAMOS GARANTIZADOS",
"nan",
"ORGANISMOS INTERNACIONALES",
"BCIE",
"BIRF",
"BID",
"FONPLATA",
"FIDA",
"CAF",
"OFID",
"FMI",
"BEI",
"nan",
"ORGANISMOS OFICIALES",
"CLUB DE PARIS",
"OTROS BILATERALES",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARÉ 2019 - $",
"PAGARÉ 2021 - CAMMESA - U$S",
"PAGARÉ 2038 - UCP",
"nan",
"AVALES",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Extraordinarios",
"nan",
"        CORTO PLAZO (1)",
"nan",
"ADELANTOS TRANSITORIOS BCRA - Ordinarios",
"nan",
"BANCA COMERCIAL",
"nan",
"PAGARÉS DEL TESORO",
"PAGARE CUT",
"nan",
"AVALES",
"nan",
"TÍTULOS PÚBLICOS",
"nan",
"LETRAS DEL TESORO",
" - Moneda nacional",
"LETRA FFRH",
"LETRA FFSIT",
"LETRA SRT",
"LETRA ANSES",
"LETES - $",
"LETRA FFDP",
"LETRA FGS",
"LETRA - $",
"LECAP",
"nan",
" - Moneda extranjera ",
"LETRA FGS - U$S"}

In [29]:
import streamlit as st
from streamlit import session_state as S
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np
from fredapi import Fred
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import requests
import urllib.parse
from bs4 import BeautifulSoup
import re
from datetime import datetime,timedelta

def get_data(id:str|list[str],start_date:str,col_list:list[str]|str|None=None):
    now=datetime.now().strftime("%Y-%m-%d")
    if isinstance(id,list):
        link="https://apis.datos.gob.ar/series/api/series?ids="
        for i in id[:-1]:link+=f"{i},"
        link+=id[-1]
    else :link=f"https://apis.datos.gob.ar/series/api/series?ids={id}"
    result = requests.get(f"{link}&start_date={start_date}&end_date={now}").json()
    df=pd.DataFrame(result["data"])
    df.set_index(0,inplace=True)
    df.index = pd.to_datetime(df.index)
    while (df.index.max()<=datetime.strptime(now, "%Y-%m-%d")):
        try:
            __=(df.index.max()+timedelta(days=1)).strftime("%Y-%m-%d")
            _=requests.get(f"{link}&start_date={__}&end_date={now}").json()
            _=pd.DataFrame(_["data"])
            _.set_index(0,inplace=True)
            _.index = pd.to_datetime(_.index)
            df=pd.concat([df,_])
        except:break
    if isinstance(id, list):
        if isinstance(col_list, list) and len(id)==len(col_list):
            names={}
            for i in range(1,len(id)+1):
                names[i]=col_list[i-1]
            df.rename(columns=names,inplace=True)
        else:
            names={}
            for i in range(1,len(id)+1):
                names[i]=result["meta"][i]["field"]["description"]
            df.rename(columns=names,inplace=True)
    else:
        if isinstance(col_list, str):
            df.rename(columns={1:col_list},inplace=True)
        else:
            df.rename(columns={1:result["meta"][1]["field"]["description"]},inplace=True)
    return df[~df.index.duplicated(keep='first')]

In [30]:
bop=pd.read_csv("His Data/his-BOP.csv",delimiter=";")
bop['Unnamed: 0'] = pd.to_datetime(bop.iloc[:, 0].values, format='%Y-%m-%d')
bop.set_index('Unnamed: 0', inplace=True)
ids=["160.2_TL_CUENNTE_0_T_22",
    "160.2_TL_CUENTAL_0_T_20",
    "160.2_TL_CUENERA_0_T_23",
    "160.2_AVOS_RERVA_0_T_15",
    "160.2_ERES_OMTOS_0_T_23",
    "82.2_ITI_2004_T_27",
    "9.2_PDPC_2004_T_30",
    "74.2_SC_0_T_15"
    ]
cols=["CC","CK","CF","VarR","Errores","ToT","PBIUSD","XN"]
data=get_data(ids,start_date="2023-10-01",col_list=cols)
data.index = pd.to_datetime(data.index, format='%Y-%m-%d')
data.reindex(columns=bop.columns)
data=pd.concat([bop,data],axis=0)
data['CF(MBP5)']=-(data['CF']-data['VarR'])

datagdp=data.copy()
for col in data.columns.to_list():
    datagdp[col]=datagdp.rolling(4).sum()[col]*100/(datagdp["PBIUSD"]*4)


ica_his=pd.read_csv("His Data/his-ica.csv",delimiter=";")
ica_his['Unnamed: 0'] = pd.to_datetime(ica_his.iloc[:, 0].values, format='%Y-%m-%d')
ica_his.set_index('Unnamed: 0', inplace=True)
ids=["74.2_IET_0_T_16",
    "74.2_IEPP_0_T_35",
    "74.2_IEMOA_0_T_48",
    "74.2_IEMOI_0_T_46",
    "74.2_IECE_0_T_35",
    "74.2_IIT_0_T_25",
    "74.2_IIBCA_0_T_32",
    "74.2_IIBI_0_T_36",
    "74.2_IICL_0_T_42",
    "74.2_IIPABC_0_T_50",
    "74.2_IIBCO_0_T_32",
    "74.2_IIVAP_0_T_49",
    "74.2_IIR_0_T_23",
    "9.2_PDPC_2004_T_30"
    ]
cols=["Expo Totales","PP","MOA","MOI","Combustibles y Energía","Impo Totales","Bienes de capital","Bienes intermedios","Combustibles y Lubricantes","Piezas y acces","Bienes de consumo","Vehículos","Resto","PBIUSD"]
ica=get_data(ids,start_date="2023-10-01",col_list=cols)
ica.index = pd.to_datetime(ica.index, format='%Y-%m-%d')
ica.reindex(columns=ica_his.columns)
ica=pd.concat([ica_his,ica],axis=0)


In [38]:
datagdp

,CC,CK,CF,VarR,Errores,ToT,PBIUSD,XN,CF(MBP5)
2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-10-01,0.655777,0.009851,0.754830,0.356186,0.089202,0.040905,93.988784,3296.276367,-1050.817112
2007-01-01,0.639421,0.009868,0.817169,1.481621,0.167881,0.041198,97.101601,3044.330117,1711.549782
...,...,...,...,...,...,...,...,...,...
2023-01-01,-0.333710,0.006895,-0.488265,-0.092557,-0.161450,0.023172,104.352538,1004.747006,2331.310018
2023-04-01,-0.507981,0.006796,-0.597936,-0.498323,-0.096752,0.020552,94.599140,-144.519995,719.402692
2023-07-01,-0.630767,0.005303,-0.744783,-0.424225,-0.119318,0.021806,99.897229,-649.925405,2076.620631
2023-10-01,-0.855667,0.005633,-1.037186,-0.862872,-0.187152,0.022043,102.434998,-1690.120794,1068.648498


In [50]:
datagdp=data.rolling(4).sum()
datagdp['PBIUSD']=data['PBIUSD']
for col in data.columns.to_list():
    if col=='PBIUSD':pass
    else: datagdp[col]=datagdp[col]/datagdp['PBIUSD']
datagdp

,CC,CK,CF,VarR,Errores,ToT,PBIUSD,XN,CF(MBP5)
2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,210067.68650,NaN,NaN
2006-04-01,NaN,NaN,NaN,NaN,NaN,NaN,237358.75360,NaN,NaN
2006-07-01,NaN,NaN,NaN,NaN,NaN,NaN,236258.06970,NaN,NaN
2006-10-01,0.026231,0.000394,0.030193,0.014247,0.003568,0.001636,247752.05240,0.050020,-0.015946
2007-01-01,0.025577,0.000395,0.032687,0.059265,0.006715,0.001648,250122.35110,0.047274,0.026578
...,...,...,...,...,...,...,...,...,...
2023-01-01,-0.013348,0.000276,-0.019531,-0.003702,-0.006458,0.000927,614792.10440,0.006822,0.015828
2023-04-01,-0.020319,0.000272,-0.023917,-0.019933,-0.003870,0.000822,683191.00710,-0.000800,0.003985
2023-07-01,-0.025231,0.000212,-0.029791,-0.016969,-0.004773,0.000872,647149.67380,-0.004013,0.012822
2023-10-01,-0.034227,0.000225,-0.041487,-0.034515,-0.007486,0.000882,627988.90816,-0.011027,0.006973


In [16]:
import requests
from bs4 import BeautifulSoup


url="https://www.doctacapital.com.ar/dashboard/bonos/cer"
response=requests.get(url)
soup=BeautifulSoup(response.text, 'html.parser')
#val=float(soup.find(class_="sell-value").get_text(strip=True).strip().replace('$', '').replace(',', '').replace('.', ''))/100
#var=float(soup.find_all(class_="percentage")[-1].get_text(strip=True).strip().replace('%', '').replace(',', ''))/100
#var="-"
#val=float(soup.find_all('strong', class_='--fourxs')[2].text.strip().replace('$', '').replace(',', ''))/100
soup

<!DOCTYPE html>
<html lang="es" translate="no"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link as="font" crossorigin="" href="/_next/static/media/c9a5bc6a7c948fb0-s.p.woff2" rel="preload" type="font/woff2"/><link data-precedence="next" href="/_next/static/css/c7c2dd289ea469f4.css" rel="stylesheet"/><link as="script" fetchpriority="low" href="/_next/static/chunks/webpack-7a720364fb7fd8cb.js" rel="preload"/><script async="" src="/_next/static/chunks/fd9d1056-74a019b11446dfd1.js"></script><script async="" src="/_next/static/chunks/938-601315fd68927526.js"></script><script async="" src="/_next/static/chunks/main-app-a70687d594b32533.js"></script><script async="" src="/_next/static/chunks/0e5ce63c-cf3059445c9bc73a.js"></script><script async="" src="/_next/static/chunks/895-a0b322fdbda8d6e8.js"></script><script async="" src="/_next/static/chunks/787-1c5677c18f75c46e.js"></script><script async="" src="/_next/static/chunks/403-42f7cc0c913

In [5]:
# PBI Real:                 3.2_OGP_D_2004_T_17
# PBI per Cap (USD):        9.2_PPCDC_2004_T_33
# Emae:                     143.3_NO_PR_2004_A_31
# Emae Campo:               11.3_ISOM_2004_M_39
# Emae Minas y Canteras:    11.3_ISD_2004_M_26
# Emae Comercio:            11.3_AGCS_2004_M_41
# Emae Act. Inmob:          11.3_SEGA_2004_M_48
# ISAC:                     33.2_ISAC_SIN_EDAD_0_M_23_56
# IPI:                      453.1_SERIE_DESEADA_0_0_24_58

from librerias import get_data
a=get_data('3.2_OGP_D_2004_T_17',start_date='2024-01-01')
a.index[-1]

Timestamp('2024-01-01 00:00:00')

In [16]:
ids=['3.2_OGP_D_2004_T_17','9.2_PPCDC_2004_T_33']
pbi=get_data(ids,start_date='2004-01-01',col_list=['PBI Real','PBI Per Cap'])
pbi

,PBI Real,PBI Per Cap
0,,
2004-01-01,475987.445560,3962.483144
2004-04-01,470146.828037,4583.918820
2004-07-01,493754.234375,4264.425431
2004-10-01,500572.272908,4417.531654
2005-01-01,515004.723594,4524.667619
...,...,...
2023-01-01,723878.793002,13442.399995
2023-04-01,706304.137023,14654.091800
2023-07-01,722836.618472,14100.944269


In [15]:
def load_actividad(end):
    his_data=pd.read_csv("His Data/his-actividad.csv",delimiter=";")
    his_data['Unnamed: 0'] = pd.to_datetime(his_data.iloc[:, 0].values, format='%Y-%m-%d')
    his_data.set_index('Unnamed: 0', inplace=True)
    # PBI Real:                 3.2_OGP_D_2004_T_17
    # PBI per Cap (USD):        9.2_PPCDC_2004_T_33
    ids=[
        "143.3_NO_PR_2004_A_31",
        "11.3_ISOM_2004_M_39",
        "11.3_ISD_2004_M_26",
        "11.3_AGCS_2004_M_41",
        "11.3_SEGA_2004_M_48",
        "453.1_SERIE_DESEADA_0_0_24_58",
        "33.2_ISAC_SIN_EDAD_0_M_23_56"
    ]
    cols=["EMAE","Campo","Minas","Comercio","Inmobiliaria","IPI","ISAC"]
    data=get_data(ids,start_date="2024-01-01",col_list=cols)
    data.index = pd.to_datetime(data.index, format='%Y-%m-%d')
    his_data.columns=data.columns
    data.reindex(columns=his_data.columns)
    data=pd.concat([his_data,data],axis=0)

    ids=['3.2_OGP_D_2004_T_17','9.2_PPCDC_2004_T_33']
    pbi=get_data(ids,start_date='2004-01-01',col_list=['PBI Real','PBI Per Cap'])

    return data, pbi

,EMAE,Campo,Minas,Comercio,Inmobiliaria,IPI,ISAC
2004-01-01,98.380000,66.000000,100.700000,92.070000,91.420000,NaN,NaN
2004-02-01,98.160000,64.930000,95.410000,91.460000,94.050000,NaN,NaN
2004-03-01,97.820000,110.670000,100.250000,109.130000,99.650000,NaN,NaN
2004-04-01,95.090000,150.070000,96.700000,99.420000,97.600000,NaN,NaN
2004-05-01,96.910000,209.390000,104.100000,100.530000,99.800000,NaN,NaN
...,...,...,...,...,...,...,...
2024-01-01,142.117559,90.834490,110.023444,137.418663,147.717619,114.597703,146.098339
2024-02-01,142.042950,72.695513,107.544893,140.886742,147.330783,114.948373,141.963859
2024-03-01,140.513690,141.969430,109.842568,144.598606,147.907275,110.992257,118.982089
2024-04-01,140.452685,198.747197,111.787736,142.732317,151.565094,110.972325,121.319669


In [41]:
import requests
from bs4 import BeautifulSoup

df,data=load_canasta(1)
data

{'ARS': [0.0692911639106725, 932.40842],
 'COP': [0.0604214503862096, 4133.25002],
 'BRL': [0.301541171878207, 5.72872],
 'MXN': [0.484520491738942, 19.17822],
 'CPL': [0.0842257220859683, 951.56002]}

In [39]:
import pandas as pd
from datetime import datetime

df

,Canasta
2000-01-01 00:00:00.000000,134.919710
2000-02-01 00:00:00.000000,132.353555
2000-03-01 00:00:00.000000,132.976860
2000-04-01 00:00:00.000000,136.128711
2000-05-01 00:00:00.000000,141.251438
...,...
2024-04-01 00:00:00.000000,388.260363
2024-05-01 00:00:00.000000,382.142221
2024-06-01 00:00:00.000000,405.234246
2024-07-01 00:00:00.000000,399.559323


In [6]:
for i in _.split(' '):
    print(i)

TypeError: 'NoneType' object is not callable

In [13]:
_.get_text().split(' ')

['933,24142',
 'ago,',
 '18:36:00',
 'UTC',
 '·',
 'Renuncia',
 'de',
 'responsabilidad']

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from plotly import graph_objects as go
from io import BytesIO
#       Tasa de Interés
url = 'https://www.stat-search.boj.or.jp/ssi/mtshtml/fm01_d_1_en.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table')
df = pd.read_html(str(tables[0]))[0]
df=df.iloc[7:]
df.columns=['Fecha','Rate']
df.Fecha=pd.to_datetime(df.Fecha,format='%Y/%m/%d')
df.set_index('Fecha',inplace=True)
df['Rate'] = df['Rate'].str.replace(',', '').astype(float)
df.dropna(inplace=True)
rate=df['Rate']

#       Inflación
url = 'https://www.stat-search.boj.or.jp/ssi/mtshtml/pr01_m_1_en.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table')
df = pd.read_html(str(tables[0]))[0]
df=df.iloc[7:][[0,1]]
df.columns=['Fecha','Inf']
df.Fecha=pd.to_datetime(df.Fecha,format='%Y/%m')
df.set_index('Fecha',inplace=True)
df=df.loc['2000-01-01':]
df['Inf'] = df['Inf'].str.replace(',', '').astype(float)
df.dropna(inplace=True)
inf=df['Inf']

#       Desempleo
url = "https://www.e-stat.go.jp/en/stat-search/file-download?statInfId=000031831365&fileKind=0"
response = requests.get(url)
response.raise_for_status()
df = pd.read_excel(BytesIO(response.content))
df=df.iloc[369:][df.columns[1:5]]
df.index=pd.date_range(start='2000-01-01', periods=len(df), freq='M')
df.columns=['1','2','3','Unemployment']
df['Unemployment'] = pd.to_numeric(df['Unemployment'], errors='coerce')
df = df.dropna(subset=['Unemployment'])
unem=df['Unemployment']

c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning:

Cannot parse header or footer so it will be ignored



In [66]:
inf.index = inf.index.to_period('M').to_timestamp('M')
inf.index

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31',
               ...
               '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31',
               '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31',
               '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='Fecha', length=295, freq='M')

In [26]:
url = 'https://www.stat-search.boj.or.jp/ssi/mtshtml/pr01_m_1_en.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table')
df = pd.read_html(str(tables[0]))[0]
df=df.iloc[7:][[0,1]]
df.columns=['Fecha','Inf']
df.Fecha=pd.to_datetime(df.Fecha,format='%Y/%m')
df.set_index('Fecha',inplace=True)
df=df.loc['2000-01-01':]
df['Inf'] = df['Inf'].str.replace(',', '').astype(float)
df.dropna(inplace=True)
fig=go.Figure()
fig.add_trace(go.Scatter(x=df.index,y=df['Inf']))
fig.show()


In [46]:
import pandas as pd
import requests
from io import BytesIO
url = "https://www.e-stat.go.jp/en/stat-search/file-download?statInfId=000031831365&fileKind=0"
response = requests.get(url)
response.raise_for_status()
df = pd.read_excel(BytesIO(response.content))
df=df.iloc[369:][df.columns[1:5]]
df.index=pd.date_range(start='2000-01-01', periods=len(df), freq='M')
df.columns=['1','2','3','Unemployment']
df['Unemployment'] = pd.to_numeric(df['Unemployment'], errors='coerce')
df = df.dropna(subset=['Unemployment'])
unem=df['Unemployment']

c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning:

Cannot parse header or footer so it will be ignored



DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31',
               ...
               '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31',
               '2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30'],
              dtype='datetime64[ns]', length=294, freq='M')

c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning:

Cannot parse header or footer so it will be ignored



,1,2,3,Unemployment
2000-01-31,1月,Jan.,NaN,4.7
2000-02-29,2月,Feb.,NaN,4.9
2000-03-31,3月,Mar.,NaN,4.9
2000-04-30,4月,Apr.,NaN,4.8
2000-05-31,5月,May,NaN,4.6
...,...,...,...,...
2024-02-29,2月,Feb.,NaN,2.6
2024-03-31,3月,Mar.,NaN,2.6
2024-04-30,4月,Apr.,NaN,2.6
2024-05-31,5月,May,NaN,2.6


In [3]:
import pandas as pd
import requests
from io import BytesIO


In [32]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="" name="viewport"/>
<title>
</title>
<script>
    dataLayer = [{ 'gtm.whitelist': ['google', 'k', 'customScripts'] }];
</script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window, document, 'script', 'dataLayer', 'GTM-M3HGHS6');</script>
<!-- End Google Tag Manager -->
<link href="/Images/Icons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/Styles/yahoo/reset-fonts-grids.css" media="screen, print, projection" rel="stylesheet" type="text/css"/>
<link href="/Styles/yahoo/base-min.css" media="screen, prin

In [47]:
from plotly import graph_objects as go

In [74]:
df

,Tasa,Hoy,Ayer,1Sem,1Mes
8,475-500,24.5,38.0%,25.0%,3.7%
9,500-525,75.5,62.0%,75.0%,88.6%
10,525-550,0.0,0.0%,0.0%,7.7%


In [80]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [84]:
url = "https://cmegroup-tools.quikstrike.net/User/QuikStrikeView.aspx?viewitemid=IntegratedFedWatchTool&userId=lwolf&jobRole=&company=&companyType=&userId=lwolf&jobRole=&company=&companyType=&insid=134330738&qsid=cd7c9839-5cb4-4b1e-884c-2e559f28b43d"
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table')
table = tables[4]

headers = []
for th in table.find_all('th'):
    headers.append(th.get_text(strip=True).replace('\n', ' '))
rows = []
for tr in table.find_all('tr', {'class': ['hide', '']}):
    cells = tr.find_all('td')
    row = [cell.get_text(strip=True) for cell in cells]
    if row:  # Ignorar filas vacías
        rows.append(row)
df = pd.DataFrame(rows, columns=headers[:len(rows[0])])
df['Now*'] = df['Now*'].replace('', '0.0%')
df['Now*'] = df['Now*'].str.rstrip('%').astype(float)
df = df[(df['Now*'] != 0.0) | (df['Target Rate (bps)'].str.contains(r'\(Current\)'))].copy()
df['Now*'] = df['Now*'].astype(str) + '%'
df.columns=['Tasa','Hoy','Ayer','1Sem','1Mes']
df['Tasa'] = df['Tasa'].str.replace(r'\(Current\)', '', regex=True).str.strip()
df['Hoy'] = df['Hoy'].str.rstrip('%').astype(float)
df['Ayer'] = df['Ayer'].str.rstrip('%').astype(float)
df['1Sem'] = df['1Sem'].str.rstrip('%').astype(float)
df['1Mes'] = df['1Mes'].str.rstrip('%').astype(float)

hoy = datetime.today()
ayer = hoy - timedelta(days=1)
una_semana_atras = hoy - timedelta(weeks=1)
un_mes_atras = hoy - relativedelta(months=1)
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df['Tasa'],
    y=df['1Mes'],
    name=un_mes_atras.strftime("%d de %b"),
    marker_color='violet',  # Color bordo
    text=df['1Mes'].apply(lambda x: f'{x:.2f}%'),  # Mostrar valores en porcentaje
    textposition='outside',
    marker=dict(cornerradius="15%",line=dict(color='purple',width=2))
))
fig.add_trace(go.Bar(
    x=df['Tasa'],
    y=df['1Sem'],
    name=una_semana_atras.strftime("%d de %b"),
    marker_color='green',  # Color bordo
    text=df['1Sem'].apply(lambda x: f'{x:.2f}%'),  # Mostrar valores en porcentaje
    textposition='outside',
    marker=dict(cornerradius="15%",line=dict(color='darkgreen',width=2))
))
fig.add_trace(go.Bar(
    x=df['Tasa'],
    y=df['Ayer'],
    name='Ayer',
    marker_color='royalblue',  # Color bordo
    text=df['Ayer'].apply(lambda x: f'{x:.2f}%'),  # Mostrar valores en porcentaje
    textposition='outside',
    marker=dict(cornerradius="15%",line=dict(color='darkblue',width=2))
))
fig.add_trace(go.Bar(
    x=df['Tasa'],
    y=df['Hoy'],
    name='Hoy',
    marker_color='crimson',  # Color bordo
    text=df['Hoy'].apply(lambda x: f'{x:.2f}%'),  # Mostrar valores en porcentaje
    textposition='outside',
    marker=dict(cornerradius="15%",line=dict(color='darkred',width=2))
))
fig.update_layout(
    plot_bgcolor='white',
    yaxis=dict(range=[0, 100],showline=True, linewidth=2, linecolor='black',gridcolor='lightslategrey',gridwidth=0.35),
    title="Distribución de la Probabilidad de la Tasa de la Fed",
    xaxis_title="Tasa objetivo (Basis Points)",
    yaxis_title="Probabilidad",
    showlegend=True)
fig.show()

In [15]:
# Extraer los encabezados de la tabla
headers = []
for th in table.find_all('th'):
    headers.append(th.get_text(strip=True).replace('\n', ' '))

# Extraer todas las filas de la tabla
rows = []
for tr in table.find_all('tr', {'class': ['hide', '']}):  # Ignora la fila que contiene el mensaje de data
    cells = tr.find_all('td')
    row = [cell.get_text(strip=True) for cell in cells]
    if row:  # Ignorar filas vacías
        rows.append(row)

# Crear un DataFrame a partir de los datos
df = pd.DataFrame(rows, columns=headers)

# Mostrar el DataFrame
print(df)

ValueError: 6 columns passed, passed data had 5 columns

In [4]:
import requests
from bs4 import BeautifulSoup

# URL de la página de CME FedWatch Tool
url = "https://www.cmegroup.com/trading/interest-rates/countdown-to-fomc.html"

# Hacer una solicitud HTTP a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar la tabla con los datos de probabilidades de la tasa de interés
table = soup.find('table', {'class': 'cmeTable'})

# Procesar los datos
data = []
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Convertir los datos a un DataFrame de pandas
import pandas as pd

df = pd.DataFrame(data, columns=headers)
df = df.dropna(how='all')  # Eliminar filas vacías

print(df)


: 

: 

In [92]:
from selenium import webdriver
import pandas as pd

driver = webdriver.Chrome()
driver.get(url)
table_xpath = "//table[@class='grid-thm grid-thm-v2 w-lg']"
table = driver.find_element("xpath", table_xpath)
headers = table.find_elements("xpath", ".//th")
header_names = [header.text.strip() for header in headers]
rows = table.find_elements("xpath", ".//tr[not(@class='compact')]")
data = []
for row in rows:
    cols = row.find_elements("xpath", ".//td")
    cols_data = [col.text.strip() for col in cols]
    if cols_data:  # Solo agregar si la fila no está vacía
        data.append(cols_data)
df = pd.DataFrame(data)
df.columns = ['Tasa','Hoy','Ayer','1Sem','1Mes']
df['Hoy'] = df['Hoy'].replace('', '0.0%')
df['Hoy'] = df['Hoy'].str.rstrip('%').astype(float)
df = df[(df['Hoy'] != 0.0) | (df['Tasa'].str.contains(r'\(Current\)'))].copy()
df['Hoy'] = df['Hoy'].astype(str) + '%'
df['Tasa'] = df['Tasa'].str.replace(r'\(Current\)', '', regex=True).str.strip()
df['Hoy'] = df['Hoy'].str.rstrip('%').astype(float)
df['Ayer'] = df['Ayer'].str.rstrip('%').astype(float)
df['1Sem'] = df['1Sem'].str.rstrip('%').astype(float)
df['1Mes'] = df['1Mes'].str.rstrip('%').astype(float)
df.dropna(inplace=True)
df.to_csv('fed_rate_data.csv', index=False)
driver.quit()

In [93]:
df=pd.read_csv('fed_rate_data.csv')
df

,Tasa,Hoy,Ayer,1Sem,1Mes
0,425-450,6.6,16.6,5.5,0.0
1,450-475,38.2,48.5,36.0,2.2
2,475-500,55.2,35.0,58.5,53.6
3,525-550,0.0,0.0,0.0,3.2


In [64]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Configuración del webdriver (asegúrate de tener el webdriver correcto instalado)
url = "https://cmegroup-tools.quikstrike.net/User/QuikStrikeView.aspx?viewitemid=IntegratedFedWatchTool&userId=lwolf&jobRole=&company=&companyType=&userId=lwolf&jobRole=&company=&companyType=&insid=134330738&qsid=cd7c9839-5cb4-4b1e-884c-2e559f28b43d"

def extract_data():
    driver = webdriver.Chrome()
    # Navega al sitio web
    driver.get(url)
    driver.implicitly_wait(10)
    # Haz clic en la pestaña "Probabilities"
    probabilities_tab = driver.find_element(By.ID, "ctl00_MainContent_ucViewControl_IntegratedFedWatchTool_lbPTree")
    probabilities_tab.click()
    driver.implicitly_wait(5)
    table = driver.find_elements(By.TAG_NAME, "table")[2]
    rows = table.find_elements(By.TAG_NAME, "tr")
    columns=[col.text for col in rows[1].find_elements(By.TAG_NAME, "th")]
    data = []
    for row in rows[2:]:
        data.append(row.text.split(' '))
    count=len(columns)
    data2=[]
    for i in range(len(data)):
        if len(data[i])<count:
            remplazar=[data[i][0]]
            for j in range(count-len(data[i])):
                remplazar.append('0.0%')
            for j in data[i][1:]:
                remplazar.append(j)
            data2.append(remplazar)
        else: data2.append(data[i])
    driver.quit()
    return data2,columns
strikes=0
while strikes!=3:
    try:
        data,columns=extract_data()
        columns = columns
        df = pd.DataFrame(data, columns=columns)
        df.to_csv('fed_rate_data.csv',index=False)
        print('Datos Obtenidos')
        break
    except:
        time.sleep(5)
        strikes+=1
        pass

Datos Obtenidos


In [195]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
driver = webdriver.Chrome()
# Navega al sitio web
driver.get(url)
driver.implicitly_wait(10)
# Haz clic en la pestaña "Probabilities"
dotplot_tab = driver.find_element(By.ID, "ctl00_MainContent_ucViewControl_IntegratedFedWatchTool_lbDotPlotTable")
dotplot_tab.click()
time.sleep(5)
table=driver.find_elements(By.TAG_NAME, "table")[1]
rows = table.find_elements(By.TAG_NAME, "tr")
column=rows[0].text.split(' ')
column.remove('RUN')
column[-1]='Largo Plazo'
column[0]=column[0].replace('\n',' ')
data=[]
time.sleep(1)
for row in rows[1:]:
    row_data=[]
    attempts = 0
    success = False
    while attempts<3:
        try:
            cols=row.find_elements(By.TAG_NAME, "td")
            for i in cols:
                row_data.append(float(i.text) if i.text!='' else None)
            success = True
            break
        except StaleElementReferenceException:
            attempts += 1
    data.append(row_data)
driver.quit()
df=pd.DataFrame(data,columns=column)
df.to_csv('dotplot.csv',index=False)

In [214]:
data=df.copy()
data.set_index('TARGET RATE',inplace=True)
data

,2024,2025,2026,Largo Plazo
TARGET RATE,,,,
5.500,NaN,NaN,NaN,NaN
5.375,4.0,1.0,NaN,NaN
5.250,NaN,NaN,NaN,NaN
5.125,7.0,NaN,NaN,NaN
5.000,NaN,NaN,NaN,NaN
4.875,8.0,1.0,1.0,NaN
4.750,NaN,NaN,NaN,NaN
4.625,NaN,NaN,NaN,NaN
4.500,NaN,NaN,NaN,NaN


In [260]:
def generar_numeros(base, numero):
    mitad = numero // 2
    if numero % 2 != 0:  # Si es impar
        numeros=[]
        for i in range(1,1+mitad):
            numeros.append(base+0.05*i)
            numeros.append(base-0.05*i)
        numeros.append(base)
        numeros.sort()
    else:  # Si es par
        numeros=[]
        for i in range(1,1+mitad):
            numeros.append(base+0.05*i)
            numeros.append(base-0.05*i)
        numeros.sort()
    return numeros


plot_dict={}

for col in data.columns:
    plot_data={}
    if col=='Largo Plazo':
        year=int(data.columns[-2])+1
    else:
        year=int(col)
    year_data=data.dropna(subset=[col])[col]
    for val in year_data.index:
        plot_data[val]=generar_numeros(int(year),int(year_data[val]))
    plot_dict[col]=plot_data

In [300]:
[k for k in plot_dict.keys()]

['2024', '2025', '2026', 'Largo Plazo']

In [309]:
# Creación de la figura
fig = go.Figure()

# Añadir datos al gráfico
for year, rates in plot_dict.items():
    for rate, votes in rates.items():
        fig.add_trace(go.Scatter(
            x=votes,
            y=[rate] * len(votes),
            mode='markers',
            marker=dict(size=10,color='navy'),
            name=str(year)
        ))

texts=[k for k in plot_dict.keys()]
vals=[(int(k) if k!="Largo Plazo" else (int(texts[-2])+1)) for k in texts]
fig.update_layout(
    title="Dot Plot del FOMC",
    plot_bgcolor='white',
    xaxis_title="Año",
    yaxis_title="Tasa de Interés (%)",
    showlegend=False,
    xaxis=dict(tickmode='array',tickvals=vals,ticktext=texts),
    yaxis=dict(range=[0,data.index.max().max()*1.1],showline=True, linewidth=0.5, linecolor='black',gridcolor='lightslategrey',gridwidth=0.35)
)

# Mostrar el gráfico
fig.show()

In [303]:
vals

[2024, 2025, 2026, 2027]

In [273]:
def valor_median(lista):
    # Ordenar la lista
    lista_ordenada = sorted(lista)
    
    # Encontrar la mediana
    n = len(lista_ordenada)
    if n % 2 == 1:
        # Lista tiene un número impar de elementos
        mediana = lista_ordenada[n // 2]
    else:
        # Lista tiene un número par de elementos
        mid1 = lista_ordenada[n // 2 - 1]
        mid2 = lista_ordenada[n // 2]
        mediana = (mid1 + mid2) / 2
    
    return mediana

# Lista de valores
valores = rates.keys()

# Encontrar el valor que corresponde a la mediana
#mediana = np.median(valores)
print(valores)
# Imprimir el resultado
print("La mediana es:", mediana)


dict_keys([3.75, 3.625, 3.5, 3.125, 3.0, 2.75, 2.625, 2.5, 2.375])
La mediana es: 3.0


In [234]:

# Ejemplos
resultado_impar = generar_numeros(2021, 1)  # Debería devolver [2014.9, 2015, 2015.1]
resultado_par = generar_numeros(2015, 4)    # Debería devolver [2014.8, 2014.9, 2015.1, 2015.2]

print("Resultado para número impar (3):", resultado_impar)
print("Resultado para número par (4):", resultado_par)


Resultado para número impar (3): [2021]
Resultado para número par (4): [2014.8, 2014.9, 2015.1, 2015.2]


In [212]:
from plotly import graph_objects as go

# Convertir el DataFrame a formato largo (long format)
df_long = df.melt(id_vars='TARGET RATE', var_name='Year', value_name='Votes')

# Filtrar filas donde no hay votos
df_long = df_long.dropna(subset=['Votes'])

# Crear el gráfico
fig = go.Figure()

# Añadir puntos para cada año
for year in df_long['Year'].unique():
    df_year = df_long[df_long['Year'] == year]
    fig.add_trace(go.Scatter(
        x=[year] * int(df_year['Votes'].sum()),  # Repetir el año según el número de votos
        y=df_year['TARGET RATE'].repeat(df_year['Votes'].astype(int)),  # Repetir la tasa según el número de votos
        mode='markers',
        name=year,
        marker=dict(size=10)
    ))

# Ajustes de diseño
fig.update_layout(
    title="FOMC Dot Plot",
    xaxis_title="Año",
    yaxis_title="Tasa de Interés (%)",
    yaxis=dict(autorange="reversed"),  # Invertir el eje Y
    template="plotly_white",
    showlegend=False
)

# Mostrar el gráfico
fig.show()

In [213]:
df_long

,TARGET RATE,Year,Votes
1,5.375,2024,4.0
3,5.125,2024,7.0
5,4.875,2024,8.0
28,5.375,2025,1.0
32,4.875,2025,1.0
36,4.375,2025,4.0
38,4.125,2025,9.0
40,3.875,2025,2.0
42,3.625,2025,1.0
48,2.875,2025,1.0


In [233]:
import plotly.graph_objs as go
import numpy as np

# Datos simulados basados en tu tabla inicial
x_2024 = np.repeat(2024, 20) + np.random.uniform(-0.2, 0.2, 20)
y_2024 = [5.375, 5.375, 5.375, 5.375, 5.375, 5.375, 5.375, 4.875, 4.875, 4.875, 
          4.125, 4.125, 4.125, 4.125, 4.125, 3.875, 3.875, 3.875, 3.875, 3.875]

x_2025 = np.repeat(2025, 20) + np.random.uniform(-0.1, 0.1, 20)
y_2025 = [5.375, 5.375, 4.875, 4.875, 4.875, 4.125, 4.125, 3.875, 3.875, 3.875, 
          3.625, 3.625, 2.875, 2.875, 2.875, 2.875, 2.875, 2.375, 2.375, 2.375]

x_2026 = np.repeat(2026, 20) + np.random.uniform(-0.1, 0.1, 20)
y_2026 = [4.875, 4.875, 4.125, 4.125, 4.125, 3.875, 3.875, 3.625, 3.625, 3.625, 
          3.375, 3.375, 3.375, 3.375, 3.125, 3.125, 2.875, 2.875, 2.875, 2.625]

x_long_term = np.repeat(2027, 10) + np.random.uniform(-0.1, 0.1, 10)
y_long_term = [3.625, 3.625, 3.625, 3.375, 3.375, 3.375, 3.125, 3.125, 2.875, 2.875]

# Combina los datos
x = np.concatenate([x_2024, x_2025, x_2026, x_long_term])
y = np.concatenate([y_2024, y_2025, y_2026, y_long_term])

# Crea el trazado
trace1 = go.Scatter(
    mode="markers",
    x=[2023.9,2024,2024.1],
    y=[1.65,1.65,1.65],
    marker=dict(size=10, color="blue")
    #text=[f'{year} : {rate}' for year, rate in zip(x, y)],
    #hoverinfo="text"
)


# Configura el layout
layout = go.Layout(
    title="FOMC Dot Plot",
    xaxis=dict(dtick=1, tick0=2024, title="Projection Year End", showticklabels=False),
    yaxis=dict(title="Implied Fed Funds Target Rate"),
    showlegend=False
)

# Crea la figura y muéstrala
fig = go.Figure(data=[trace1], layout=layout)
fig.show()


In [60]:
print(len(columns))
for i in range(len(data2)):
    print(len(data2[i]))

13
13
13
13
13
13
13
13
13
13


In [93]:
from datetime import datetime
df=pd.read_csv('fed_rate_data.csv')
for i in df.columns[1:]:
    df[i] = df[i].str.rstrip('%').astype(float)
df['MEETING DATE']=pd.to_datetime(df['MEETING DATE'],format='%m/%d/%Y')
df.set_index('MEETING DATE',inplace=True)
df.index=df.index.strftime('%d de %b %Y')
df=df.transpose()
df

MEETING DATE,18 de Sep 2024,07 de Nov 2024,18 de Dec 2024,29 de Jan 2025,19 de Mar 2025,07 de May 2025,18 de Jun 2025,30 de Jul 2025,17 de Sep 2025
225-250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.8
250-275,0.0,0.0,0.0,0.0,0.0,0.0,0.2,2.1,4.9
275-300,0.0,0.0,0.0,0.0,0.0,0.3,2.4,10.6,15.8
300-325,0.0,0.0,0.0,0.0,0.5,3.8,11.9,26.7,29.0
325-350,0.0,0.0,0.0,0.2,5.2,17.6,28.9,33.8,29.6
350-375,0.0,0.0,0.0,3.5,22.7,36.9,34.5,20.7,15.8
375-400,0.0,0.0,2.6,20.4,42.8,32.9,18.5,5.5,3.9
400-425,0.0,0.0,18.9,44.1,28.7,8.5,3.5,0.5,0.3
425-450,0.0,6.8,44.7,31.9,0.0,0.0,0.0,0.0,0.0
450-475,0.0,38.5,33.9,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
col='18 de Sep 2024'
df[df[col]>0][col].index

Index(['475-500', '500-525'], dtype='object')